## Simulating melts of polymer chains

In [4]:
import warnings
warnings.filterwarnings('ignore')
import hoomd
import gsd
import gsd.hoomd
from hoomd.md import nlist

import unyt as u
import glob
import mbuild as mb
import matplotlib.pyplot as plt
import numpy as np
import os

import flowermd
from flowermd.utils import get_target_box_mass_density
from flowermd.base import Simulation, Molecule
from flowermd.library import FF_from_file
from flowermd.base.system import Pack

from polymer_dictionary import polymer_dictionary

# Rerun with the original seed first

In [2]:
# Updates atom names to fit with hoomd conventions
def espaloma_mol(file_path):
     mol = mb.load(file_path)
     for p in mol.particles():
           p.name = f"_{p.name}"
     return mol

## Single molecule sim:

In [3]:
key_list = sorted(list(polymer_dictionary.keys())) # Aligning dictionary and path to aid automation
path = os.getcwd() # Non-user specific path
molecule_list = sorted(glob.glob(path+"/mol2/*.mol2"))
molecule_list.pop(10) # Removing polymer not tested
print(key_list)

['PCPDTFBT_C11_BO', 'PCPDTFBT_C1_BO', 'PCPDTFBT_C3_BO', 'PCPDTFBT_C4_BO', 'PCPDTFBT_C5_BO', 'PCPDTPT_HD', 'PCPDTPT_ODD', 'PCPDTPT_eneODD', 'PCPDTPT_nC16', 'PCPDT_PT_eneHD', 'PIDTBT_nC16', 'PIDTCPDT_C11BO', 'PIDTFBT_C11_BO']


In [4]:
mol2_path = os.getcwd() + "/mol2/"
xml_path = os.getcwd() + "/xml/"

In [15]:
system_file = mol2_path + "alternate_PCPDTPT_nC16.mol2"
ff_filepath = xml_path + "alternate_PCPDTPT_nC16.xml"

espmol = espaloma_mol(system_file)
molecule = Molecule(num_mols=1, compound=espmol)

molff = FF_from_file(ff_filepath)
system = Pack(molecules=molecule,density=0.01 * u.g/u.cm**3,packing_expand_factor=5)
system.apply_forcefield(r_cut=2.5, force_field=molff, auto_scale=True,remove_charges=True, remove_hydrogens=True)

No charged group detected, skipping electrostatics.


In [16]:
system.hoomd_snapshot
hoomd_forces = system.hoomd_forcefield
hoomd_forces
lj_force = hoomd_forces[3]
cpu = hoomd.device.CPU()
sim = Simulation.from_system(system=system, gsd_write_freq=1, log_write_freq=1, device=cpu, gsd_file_name="alternate_PCPDTPT_nC16_melt.gsd",log_file_name="alternate_PCPDTPT_nC16_melt.txt")

Initializing simulation state from a gsd.hoomd.Frame.


In [17]:
sim.run_NVT(n_steps=1, kT=5.0, tau_kt=1.0)
sim.flush_writers()

In [24]:
sim.reference_values

{'energy': unyt_quantity(1.046, 'kJ/mol'),
 'length': unyt_quantity(0.35635949, 'nm'),
 'mass': unyt_quantity(32.06, 'amu')}

In [40]:
energy = ((1046/6.022*10**23)*J).to('J')
length = (0.35635949*nm).to('m')
mass = (32.06*amu).to('kg')

In [41]:
tau = np.sqrt((mass*length**2)/energy)

In [45]:
print(tau)

1.9728730976366816e-35 sqrt(kg)*m/sqrt(J)


## Running sims of every polymer in the Danielsen et al. paper

In [5]:
pack_seeds = [1300,1400,1500]

In [11]:
for pack_seed in pack_seeds:
    for i in range(len(key_list)):
        system_file = mol2_path + "10_mers/" + key_list[i] + "_10mer.mol2"
        ff_filepath = xml_path + key_list[i] + ".xml"
        gsd_path = os.getcwd() + "/gsd_files/10_mers/"+str(pack_seed)+"/"
        num_mers = "_10mer" # The number of monomers being simulated is determined by the mol2, this is for naming the resulting gsd and .txt files
        espmol = espaloma_mol(system_file)
        molecule = Molecule(num_mols=1, compound=espmol)
        cpu = hoomd.device.CPU()
        
        molff = FF_from_file(ff_filepath)
        system = Pack(molecules=molecule,density=0.01 * u.g/u.cm**3,seed=pack_seed, packing_expand_factor=5)
        system.apply_forcefield(r_cut=2.5, force_field=molff, auto_scale=True,remove_charges=True, remove_hydrogens=True)
        system.hoomd_snapshot
        hoomd_forces = system.hoomd_forcefield
        hoomd_forces
        lj_force = hoomd_forces[3]
        sim = Simulation.from_system(system=system, gsd_write_freq=5000, log_write_freq=5000, device=cpu, gsd_file_name=gsd_path+key_list[i]+num_mers+"_seed_"+str(pack_seed)+".gsd",log_file_name=gsd_path+key_list[i]+num_mers+"_seed_"+str(pack_seed)+".txt")
        target_box = flowermd.utils.get_target_box_mass_density(
                density=0.05 * u.g/u.cm**3, mass=sim.mass.to("g"))
        sim.run_update_volume(n_steps=5000, kT=5.0, tau_kt=1.0, final_box_lengths=target_box, period=10)
        sim.run_NVT(n_steps=5e5, kT=5.0, tau_kt=1.0)
        sim.flush_writers()

No charged group detected, skipping electrostatics.
Initializing simulation state from a gsd.hoomd.Frame.
Step 5000 of 5000; TPS: 457.37; ETA: 0.0 minutes
Step 4999 of 500000; TPS: 707.46; ETA: 11.7 minutes
Step 9999 of 500000; TPS: 671.9; ETA: 12.2 minutes
Step 14999 of 500000; TPS: 728.3; ETA: 11.1 minutes
Step 19999 of 500000; TPS: 766.2; ETA: 10.4 minutes
Step 24999 of 500000; TPS: 788.43; ETA: 10.0 minutes
Step 29999 of 500000; TPS: 801.77; ETA: 9.8 minutes
Step 34999 of 500000; TPS: 810.26; ETA: 9.6 minutes
Step 39999 of 500000; TPS: 781.3; ETA: 9.8 minutes
Step 44999 of 500000; TPS: 791.53; ETA: 9.6 minutes
Step 49999 of 500000; TPS: 798.44; ETA: 9.4 minutes
Step 54999 of 500000; TPS: 803.32; ETA: 9.2 minutes
Step 59999 of 500000; TPS: 804.84; ETA: 9.1 minutes
Step 64999 of 500000; TPS: 779.87; ETA: 9.3 minutes
Step 69999 of 500000; TPS: 777.16; ETA: 9.2 minutes
Step 74999 of 500000; TPS: 777.72; ETA: 9.1 minutes
Step 79999 of 500000; TPS: 778.1; ETA: 9.0 minutes
Step 84999 of 5

In [13]:
from flowermd.library import KremerGrestBeadSpring, LJChain
from flowermd.library.forcefields import BeadSpring

In [5]:
kg_chain = LJChain(lengths=100,num_mols=1)
cpu = hoomd.device.CPU()

In [11]:
system = Pack(molecules=kg_chain, 
              density=0.00001)

In [14]:
ff = BeadSpring(
    r_cut=2**(1/6),  
    beads={
        "A": dict(epsilon=1.0, sigma=1.0),  # chains
        "F": dict(epsilon=1.0, sigma=1.0),  # flakes
    },
    bonds={
        "F-F": dict(r0=1.0, k=1000),
        "A-A": dict(r0=1.0, k=1000.0),  # increased k to avoid chain collapse
    },
    angles={
        "A-A-A": dict(t0=2* np.pi / 3., k=100.0),   # moderate stiffness for chains
        "F-F-F": dict(t0=2 * np.pi / 3., k=5000),
    },
    dihedrals={
        "A-A-A-A": dict(phi0=0.0, k=0, d=-1, n=2), #need to turn this on later, messed up with straight chains
        "F-F-F-F": dict(phi0=0.0, k=500, d=-1, n=2),
    }
)

In [15]:
sim = Simulation(initial_state=system.hoomd_snapshot, forcefield=ff.hoomd_forces, device=cpu, gsd_write_freq=int(1000), log_file_name="test1.txt", gsd_file_name="test1.gsd")
sim.run_NVT(n_steps=5e5, kT=7, tau_kt=1)
sim.flush_writers()

*Warning*: dihedral.harmonic: specified K <= 0


Initializing simulation state from a gsd.hoomd.Frame.
Step 1000 of 500000; TPS: 1906.67; ETA: 4.4 minutes
Step 2000 of 500000; TPS: 3675.15; ETA: 2.3 minutes
Step 3000 of 500000; TPS: 5332.63; ETA: 1.6 minutes
Step 4000 of 500000; TPS: 6886.38; ETA: 1.2 minutes
Step 5000 of 500000; TPS: 8355.94; ETA: 1.0 minutes
Step 6000 of 500000; TPS: 9747.19; ETA: 0.8 minutes
Step 7000 of 500000; TPS: 11057.54; ETA: 0.7 minutes
Step 8000 of 500000; TPS: 12301.39; ETA: 0.7 minutes
Step 9000 of 500000; TPS: 13467.69; ETA: 0.6 minutes
Step 10000 of 500000; TPS: 14591.06; ETA: 0.6 minutes
Step 11000 of 500000; TPS: 15663.05; ETA: 0.5 minutes
Step 12000 of 500000; TPS: 16688.41; ETA: 0.5 minutes
Step 13000 of 500000; TPS: 17619.64; ETA: 0.5 minutes
Step 14000 of 500000; TPS: 18459.57; ETA: 0.4 minutes
Step 15000 of 500000; TPS: 19284.94; ETA: 0.4 minutes
Step 16000 of 500000; TPS: 20118.57; ETA: 0.4 minutes
Step 17000 of 500000; TPS: 20875.42; ETA: 0.4 minutes
Step 18000 of 500000; TPS: 21639.92; ETA: 0